In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
np.random.seed(0)
tf.random.set_seed(0)

2023-09-15 23:46:25.486188: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 23:46:25.553383: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 23:46:25.554565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 23:46:26.545510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('train.csv')
df.head()

,date_time,pressure
0,2023-01-01 00:00:00,0.595000
1,2023-01-01 01:00:00,0.582547
2,2023-01-01 02:00:00,0.548851
3,2023-01-01 03:00:00,0.503720
4,2023-01-01 04:00:00,0.460289


In [3]:
# feature engineering the time series data
df.date_time = pd.to_datetime(df.date_time)
df['year'] = df.date_time.dt.year
df['month'] = df.date_time.dt.month
df['day'] = df.date_time.dt.day
df['hour'] = df.date_time.dt.hour
df['day_of_week'] = df.date_time.dt.dayofweek
df['weekend'] = df.date_time.dt.weekday // 4
df.drop(['date_time'], axis=1, inplace=True)
df.head()

,pressure,year,month,day,hour,day_of_week,weekend
0,0.595000,2023,1,1,0,6,1
1,0.582547,2023,1,1,1,6,1
2,0.548851,2023,1,1,2,6,1
3,0.503720,2023,1,1,3,6,1
4,0.460289,2023,1,1,4,6,1


In [4]:
# Normalizing the data
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df_scaled = scaler.fit_transform(df)
# df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
# df_scaled.shape

(8737, 7)

In [5]:
model = Sequential([
    Input(shape=(df.shape[1],)),
    Dense(32, activation='relu'),
    Dense(8, activation='relu'),
    Dense(32, activation='relu'),
    Dense(df.shape[1])
])

model.compile(
    optimizer='adam',
    loss='mse'
)

In [6]:
model.fit(
    x=df,
    y=df,
    epochs=100,
    batch_size=32,
    callbacks=EarlyStopping(
        monitor='loss',
        patience=4,
        mode='min'
    )
)

Epoch 1/100
274/274 [==============================] - 1s 2ms/step - loss: 96028.4375
Epoch 2/100
274/274 [==============================] - 0s 2ms/step - loss: 23.0052
Epoch 3/100
274/274 [==============================] - 1s 2ms/step - loss: 22.5829
Epoch 4/100
274/274 [==============================] - 0s 2ms/step - loss: 22.9362
Epoch 5/100
274/274 [==============================] - 0s 2ms/step - loss: 23.1506
Epoch 6/100
274/274 [==============================] - 0s 2ms/step - loss: 22.6242
Epoch 7/100
274/274 [==============================] - 0s 2ms/step - loss: 22.7498


In [7]:
pred = model.predict(df)
error = pred - df

274/274 [==============================] - 0s 1ms/step


In [8]:
q1 = np.quantile(error.pressure, 1/4)
q3 = np.quantile(error.pressure, 3/4)
iqr = q3 - q1
lower_bound = q1 - 1.0 * iqr
upper_bound = q3 + 1.0 * iqr

is_outlier = lambda x: x < lower_bound or x > upper_bound
train_outliers = [x for x in error.pressure if is_outlier(x)]

print("Outliers in training: ", len(train_outliers), 'out of', len(error.pressure))

Outliers in training:  296 out of 8737


In [18]:
model.save('autoencoder.keras')

In [10]:
# Simulate a leak in the pipe

# Reduce the pressure by 10% for 2nd and 3rd month
leaked_pipe = df.copy()
leaked_pipe.loc[2*24*30:4*24*30, 'pressure'] *= 0.5

# Get predictions
new_pred = model.predict(leaked_pipe)
new_error = new_pred - leaked_pipe

new_outliers = [x for x in new_error.pressure if is_outlier(x)]

print('Leak detected' if len(new_outliers) > len(train_outliers) else 'No leak detected')

 36/274 [==>...........................] - ETA: 0s

274/274 [==============================] - 0s 2ms/step
Leak detected


In [11]:
len(new_outliers)

1737

In [12]:
random_point = df.iloc[45, :].copy()
random_point.pressure = random_point.pressure * 0.5
random_point = random_point.values.reshape(1, -1)
random_pred = model.predict(random_point)
random_error = random_pred - random_point
is_outlier(random_error[0, 0])

1/1 [==============================] - 0s 70ms/step


True

In [17]:
def preprocess(date_time: str, pressure: float):
    date_time = pd.to_datetime(date_time)
    year = date_time.year
    month = date_time.month
    day = date_time.day
    hour = date_time.hour
    day_of_week = date_time.dayofweek
    weekend = date_time.weekday() // 4

    return np.array([pressure, year, month, day, hour, day_of_week, weekend]).reshape(1, -1)

def check_if_outlier(date_time: str, pressure: float, model: Sequential, is_outlier: callable):
    data = preprocess(date_time, pressure)
    pred = model.predict(data)
    error = pred - data
    return is_outlier(error[0, 0])
check_if_outlier('2023-01-01 00:00:00', 0.2, model, is_outlier)

1/1 [==============================] - 0s 30ms/step


True

In [19]:
lower_bound, upper_bound

(0.35750232411031924, 0.659694164843159)